In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim

from torchpack.utils.config import configs

from torchquantum.datasets import MNIST
from torch.optim.lr_scheduler import CosineAnnealingLR

import random
import numpy as np

In [2]:
def train(dataflow, model, device, optimizer):
    for feed_dict in dataflow['train']:
        inputs = feed_dict['image'].to(device)
        targets = feed_dict['digit'].to(device)

        outputs = model(inputs)
        loss = F.nll_loss(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"loss: {loss.item()}", end='\r')

In [3]:
def valid_test(dataflow, split, model, device, qiskit=False):
    target_all = []
    output_all = []
    with torch.no_grad():
        for feed_dict in dataflow[split]:
            inputs = feed_dict['image'].to(device)
            targets = feed_dict['digit'].to(device)

            outputs = model(inputs)

            target_all.append(targets)
            output_all.append(outputs)
        target_all = torch.cat(target_all, dim=0)
        output_all = torch.cat(output_all, dim=0)

    _, indices = output_all.topk(1, dim=1)
    masks = indices.eq(target_all.view(-1, 1).expand_as(indices))
    size = target_all.shape[0]
    corrects = masks.sum().item()
    accuracy = corrects / size
    loss = F.nll_loss(output_all, target_all).item()

    print(f"{split} set accuracy: {accuracy}")
    print(f"{split} set loss: {loss}")

In [4]:
from q4digit_models import Q4DigitFCModel0

In [15]:
def main(epochs=5):

    seed = 0
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    dataset = MNIST(
        root='./mnist_data',
        train_valid_split_ratio=[0.9, 0.1],
        digits_of_interest=[0,1,2,3],
        n_test_samples=100,
        n_train_samples=500,
        fashion=True
    )
    dataflow = dict()

    for split in dataset:
        sampler = torch.utils.data.RandomSampler(dataset[split])
        dataflow[split] = torch.utils.data.DataLoader(
            dataset[split],
            batch_size=10,
            sampler=sampler,
            num_workers=8,
            pin_memory=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    configs.load("./fashion4.yml", recursive=True)
    configs.model.arch.n_blocks = 8

    model = Q4DigitFCModel0(arch=configs.model.arch).to(device)

    n_epochs = epochs
    optimizer = optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)

    for epoch in range(1, n_epochs + 1):
        # train
        print(f"Epoch {epoch}:")
        train(dataflow, model, device, optimizer)
        print(optimizer.param_groups[0]['lr'])

        # valid
        valid_test(dataflow, 'valid', model, device)
        scheduler.step()

    # test
    valid_test(dataflow, 'test', model, device, qiskit=False)

    return model


In [16]:
model  = main(15)

[2023-05-02 17:40:26.034] Only use the front 500 images as TRAIN set.
[2023-05-02 17:40:26.144] Only use the front 100 images as TEST set.


Epoch 1:
0.005 1.1090143918991089
valid set accuracy: 0.7229166666666667
valid set loss: 1.150026798248291
Epoch 2:
0.0049453690018345145215
valid set accuracy: 0.74625
valid set loss: 1.083186388015747
Epoch 3:
0.0047838636441065028604
valid set accuracy: 0.7583333333333333
valid set loss: 1.0395963191986084
Epoch 4:
0.0045225424859373685206
valid set accuracy: 0.71
valid set loss: 1.0277777910232544
Epoch 5:
0.0041728265158971455192
valid set accuracy: 0.7516666666666667
valid set loss: 1.010713815689087
Epoch 6:
0.00375.0349903106689453
valid set accuracy: 0.7558333333333334
valid set loss: 0.9983078241348267
Epoch 7:
0.0032725424859373687783
valid set accuracy: 0.7820833333333334
valid set loss: 0.9919663667678833
Epoch 8:
0.0027613211581691348452
valid set accuracy: 0.7566666666666667
valid set loss: 0.9929884672164917
Epoch 9:
0.0022386788418308670298
valid set accuracy: 0.7645833333333333
valid set loss: 0.9873986840248108
Epoch 10:
0.0017274575140626320793
valid set accuracy: 0

In [18]:
from qdp import evaluate
from torchquantum.plugins import tq2qiskit
from qiskit2cirq import qiskit2cirq

q_layer_circ = tq2qiskit(model.q_device, model.q_layer)
circuit = qiskit2cirq(q_layer_circ)

In [19]:
evaluate(circuit, [np.array([[0,0],[0,1]]), np.array([[1,0],[0,0]])])

depolarize, 0.01
3.294563055038452
depolarize, 0.001
3.1790144443511963
depolarize, 0.0001
3.226712465286255
depolarize, 1e-05
3.2665882110595703
depolarize, 1e-06
3.214967727661133
bit_flip, 0.01
3.2624990940093994
bit_flip, 0.001
3.27093505859375
bit_flip, 0.0001
3.297316074371338
bit_flip, 1e-05
3.263076066970825
bit_flip, 1e-06
3.261585235595703
+------------+---------+----------+------+
& noise type & noisy p & kappa    & time &
+------------+---------+----------+------+
& depolarize & 0.01    & 1.052    & 3.29 &
& depolarize & 0.001   & 5.398    & 3.18 &
& depolarize & 0.0001  & 53.341   & 3.23 &
& depolarize & 1e-05   & 533.000  & 3.27 &
& depolarize & 1e-06   & 5225.775 & 3.21 &
& bit_flip   & 0.01    & 1.057    & 3.26 &
& bit_flip   & 0.001   & 5.635    & 3.27 &
& bit_flip   & 0.0001  & 56.156   & 3.30 &
& bit_flip   & 1e-05   & 561.962  & 3.26 &
& bit_flip   & 1e-06   & 5617.503 & 3.26 &
+------------+---------+----------+------+
